# Data Analysis

Objective: Quantitative analyse of data ...

In [1]:
import sqlite3, pandas as pd

In [25]:
con = sqlite3.connect("../cv-node-data/db/cvnode.db")
cur = con.cursor()

In [27]:
jobs = pd.read_sql_query("SELECT rowid, * from jobs", con, parse_dates=['begin_at', 'final_at'])
jobs.shape

(18, 7)

In [29]:
jobs.sort_values(by='rowid', ascending=False).head()

,rowid,begin_at,project_id,collect_id,thing_id,thing_tag,final_at
17,18,2025-01-13 17:13:27.044643,PR-9,CL-9,XY00-9,TG-00,2025-01-13 17:13:28.038557
16,17,2025-01-13 17:08:26.965457,PR-9,CL-9,XY00-9,TG-00,2025-01-13 17:08:29.522000
15,16,2025-01-13 17:07:38.801367,PR-9,CL-9,XY00-9,TG-00,2025-01-13 17:07:41.569918
14,15,2025-01-13 17:06:46.536195,PR-9,CL-9,XY00-9,TG-00,2025-01-13 17:06:50.603570
13,14,2025-01-13 17:05:23.399669,PR-9,CL-9,XY00-9,TG-00,2025-01-13 17:05:26.831629


In [31]:
jobs['duration'] = jobs['final_at'] - jobs['begin_at']
jobs.sort_values(by='rowid', ascending=False).head()

,rowid,begin_at,project_id,collect_id,thing_id,thing_tag,final_at,duration
17,18,2025-01-13 17:13:27.044643,PR-9,CL-9,XY00-9,TG-00,2025-01-13 17:13:28.038557,0 days 00:00:00.993914
16,17,2025-01-13 17:08:26.965457,PR-9,CL-9,XY00-9,TG-00,2025-01-13 17:08:29.522000,0 days 00:00:02.556543
15,16,2025-01-13 17:07:38.801367,PR-9,CL-9,XY00-9,TG-00,2025-01-13 17:07:41.569918,0 days 00:00:02.768551
14,15,2025-01-13 17:06:46.536195,PR-9,CL-9,XY00-9,TG-00,2025-01-13 17:06:50.603570,0 days 00:00:04.067375
13,14,2025-01-13 17:05:23.399669,PR-9,CL-9,XY00-9,TG-00,2025-01-13 17:05:26.831629,0 days 00:00:03.431960


In [44]:
jobs.describe()

,rowid,duration
count,18.000000,18
mean,9.500000,0 days 00:00:02.851874888
std,5.338539,0 days 00:00:01.887093129
min,1.000000,0 days 00:00:00.993914
25%,5.250000,0 days 00:00:01.506138250
50%,9.500000,0 days 00:00:01.960996
75%,13.750000,0 days 00:00:03.908521250
max,18.000000,0 days 00:00:07.288112


## Run

In [33]:
runs = pd.read_sql_query("SELECT rowid, * from runs", con, parse_dates=['begin_at', 'final_at'])
runs.shape

(667, 5)

In [35]:
runs.sort_values(by='rowid', ascending=False).head()

,rowid,begin_at,final_at,sensor,job_id
666,667,2025-01-13 17:13:27.942887,2025-01-13 17:13:28.030079,MaxSense - A075v,18
665,666,2025-01-13 17:13:27.871126,2025-01-13 17:13:27.939108,MaxSense - A075v,18
664,665,2025-01-13 17:13:27.802949,2025-01-13 17:13:27.867330,MaxSense - A075v,18
663,664,2025-01-13 17:13:27.732896,2025-01-13 17:13:27.799378,MaxSense - A075v,18
662,663,2025-01-13 17:13:27.663223,2025-01-13 17:13:27.729044,MaxSense - A075v,18


In [48]:
runs_1 = runs.merge(
    jobs[['rowid', 'project_id', 'collect_id', 'thing_id', 'thing_tag']], 
    left_on='job_id', 
    right_on='rowid', 
    how='left').rename(columns={'rowid_x':'run_id'}).drop(columns=['rowid_y'])
runs_1.shape

(667, 9)

In [49]:
runs_1.sort_values(by='run_id', ascending=False).head()

,run_id,begin_at,final_at,sensor,job_id,project_id,collect_id,thing_id,thing_tag
666,667,2025-01-13 17:13:27.942887,2025-01-13 17:13:28.030079,MaxSense - A075v,18,PR-9,CL-9,XY00-9,TG-00
665,666,2025-01-13 17:13:27.871126,2025-01-13 17:13:27.939108,MaxSense - A075v,18,PR-9,CL-9,XY00-9,TG-00
664,665,2025-01-13 17:13:27.802949,2025-01-13 17:13:27.867330,MaxSense - A075v,18,PR-9,CL-9,XY00-9,TG-00
663,664,2025-01-13 17:13:27.732896,2025-01-13 17:13:27.799378,MaxSense - A075v,18,PR-9,CL-9,XY00-9,TG-00
662,663,2025-01-13 17:13:27.663223,2025-01-13 17:13:27.729044,MaxSense - A075v,18,PR-9,CL-9,XY00-9,TG-00


In [22]:
r_dur = runs_1.query('final_at.notna()').groupby('job_id').agg({'begin_at':'min', 'final_at':'max'}).reset_index()
r_dur.head()

,job_id,begin_at,final_at
0,4,2025-01-13 15:44:56.102416,2025-01-13 15:45:00.870507


## Itens

In [46]:
itens = pd.read_sql_query("SELECT rowid, * from itens", con)
itens.sort_values(by='rowid', ascending=False).head()

,rowid,status,type,pov,res,file_path,run_id
1338,1339,STAGED,IMAGE,TOP_DOWN,_320_240_1,TG-00_667_2025_01_13_17_13_27_946646_STATUS_32...,667
1337,1338,STAGED,IMAGE,TOP_DOWN,_320_240_1,TG-00_667_2025_01_13_17_13_27_946646_IR_320_24...,667
1336,1337,STAGED,IMAGE,TOP_DOWN,_320_240_1,TG-00_667_2025_01_13_17_13_27_946646_DEPTH_320...,667
1335,1336,STAGED,IMAGE,TOP_DOWN,_640_480_3,TG-00_667_2025_01_13_17_13_27_946646_RGB_640_4...,667
1334,1335,STAGED,IMAGE,TOP_DOWN,_320_240_1,TG-00_666_2025_01_13_17_13_27_874840_STATUS_32...,666


In [47]:
itens.groupby('run_id').size().describe()

count    442.000000
mean       3.029412
std        1.405059
min        1.000000
25%        1.000000
50%        4.000000
75%        4.000000
max        4.000000
dtype: float64

## Data
$t = jobs * runs-per-job * items$

In [66]:
t = 23 * 265 * 4
print(f'{t} itens')

24380 itens
